In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


def load_data_from_csv(filename):
    return pd.read_csv(filename)

def load_data_from_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return pd.DataFrame(data)



data = {
    'Region': ['East', 'West', 'East', 'West', 'East', None],
    'Sales': [1000, 2000, np.nan, 3000, 1500, 1200],
    'Date': ['2023-01-01', '2023-01-02', '2023/01/03', '2023-01-04', '2023-01-05', '2023-01-06']
}
df = pd.DataFrame(data)

# Functional
def functional_pipeline(dataframe, fill_defaults={'Region': 'Unknown', 'Sales': 0}, growth_factor=1.1, save_to_file=False):
    
    def fill_missing(df):
        new_df = []
        for _, row in df.iterrows():
            new_row = {
                'Region': row['Region'] if pd.notnull(row['Region']) else fill_defaults['Region'],
                'Sales': row['Sales'] if pd.notnull(row['Sales']) else fill_defaults['Sales'],
                'Date': row['Date']
            }
            new_df.append(new_row)
        return pd.DataFrame(new_df)

    
    def standardize_dates(df):
        new_df = []
        for _, row in df.iterrows():
            try:
                row['Date'] = pd.to_datetime(row['Date']).strftime('%Y-%m-%d')
            except Exception:
                row['Date'] = None
            new_df.append(row)
        return pd.DataFrame(new_df)

   
    def filter_sales(df):
        return pd.DataFrame([row for _, row in df.iterrows() if row['Sales'] > 1000])

    
    def add_sales_growth(df):
        new_df = []
        for _, row in df.iterrows():
            row['Sales Growth'] = row['Sales'] * growth_factor
            new_df.append(row)
        return pd.DataFrame(new_df)

    
    def aggregate_data(df):
        regions = {}
        for _, row in df.iterrows():
            region = row['Region']
            if region not in regions:
                regions[region] = {'Sales': [], 'count': 0}
            regions[region]['Sales'].append(row['Sales'])
            regions[region]['count'] += 1
        results = []
        for region, stats in regions.items():
            total_sales = sum(stats['Sales'])
            average_sales = total_sales / stats['count']
            results.append({'Region': region, 'Total Sales': total_sales, 'Average Sales': average_sales})
        return pd.DataFrame(results)

    # Compose all functions
    transformed_df = fill_missing(dataframe)
    transformed_df = standardize_dates(transformed_df)
    transformed_df = filter_sales(transformed_df)
    transformed_df = add_sales_growth(transformed_df)
    aggregated_df = aggregate_data(transformed_df)

    if save_to_file:
        save_data_to_csv(transformed_df, 'functional_transformed.csv')
        save_data_to_csv(aggregated_df, 'functional_aggregated.csv')

    return transformed_df, aggregated_df

# Imperatuve
def imperative_pipeline(dataframe, fill_defaults={'Region': 'Unknown', 'Sales': 0}, growth_factor=1.1, save_to_file=False):
    
    dataframe = dataframe.copy()
    dataframe['Region'] = dataframe['Region'].fillna(fill_defaults['Region'])
    dataframe['Sales'] = dataframe['Sales'].fillna(fill_defaults['Sales'])

    
    dataframe['Date'] = pd.to_datetime(dataframe['Date'], errors='coerce')

    
    filtered_df = dataframe[dataframe['Sales'] > 1000].copy()

    
    filtered_df['Sales Growth'] = filtered_df['Sales'] * growth_factor

    
    aggregated_df = (
        filtered_df.groupby('Region')['Sales']
        .agg(['sum', 'mean'])
        .reset_index()
        .rename(columns={'sum': 'Total Sales', 'mean': 'Average Sales'})
    )

    if save_to_file:
        save_data_to_csv(filtered_df, 'imperative_transformed.csv')
        save_data_to_csv(aggregated_df, 'imperative_aggregated.csv')

    return filtered_df, aggregated_df


def plot_data(aggregated_df):
    plt.figure(figsize=(10, 6))

    # Bar chart for Total Sales
    plt.bar(aggregated_df['Region'], aggregated_df['Total Sales'], color='skyblue')
    plt.xlabel('Region')
    plt.ylabel('Total Sales')
    plt.title('Total Sales by Region')
    plt.show()


def save_data_to_csv(dataframe, filename):
    dataframe.to_csv(filename, index=False)


print("Functional Programming Paradigm:")
transformed_df_func, aggregated_df_func = functional_pipeline(df, save_to_file=True)
print(aggregated_df_func)

print("\nImperative Programming Paradigm:")
transformed_df_imp, aggregated_df_imp = imperative_pipeline(df, save_to_file=True)
print(aggregated_df_imp)

# View charts
print("\nPlotting Functional Results:")
plot_data(aggregated_df_func)

print("\nPlotting Imperative Results:")
plot_data(aggregated_df_imp)
